In [1]:
#import
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import json
import cv2
import math
import mediapipe as mp
import time
import datetime
import pyttsx3

In [8]:
IMAGE_SIZE = 255

classes_face = ['LookLeft', 'LookRight', 'LookUp', 'Neutral', 'RaiseEyebrows', 'Smile', 'OpenMouth']

input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)

model_name = "Okay model for demo"


In [5]:
# print(input_shape)

In [9]:
# try:
#     model.load_weights("./checkpoint_/" + model_name)
 
# except:
#     print("no model data")
model_filepath = "./model/"+model_name
model_face = tf.keras.models.load_model(model_filepath)
model_hand = tf.keras.models.load_model('mp_hand_gesture')


In [5]:
# print(classes)

In [10]:
# Data Processing and model code
# import necessary packages

# initialize mediapipe
mpHands = mp.solutions.hands
mpFace = mp.solutions.face_detection
hands = mpHands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

#Control variables:
show_face_mesh = False
show_hand_lm = False
predict_hand = False
predict_face = True



# Load class names
f = open('gesture.names', 'r')
classNames_hand = f.read().split('\n')
f.close()

classNames_face = classes_face
# print(classNames)

EXTRA_PADDING=0.02
gestures = []

# Initialize the webcam
drawing_spec = mpDraw.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, frame = cap.read()
        

        if not success:
          print("Ignoring empty camera frame.")
          # If loading a video, use 'break' instead of 'continue'.
          continue
        # Read each frame from the webcam

#         display(frame)
        x, y, c = frame.shape
        dimensions = frame.shape
        height = frame.shape[0]
        width = frame.shape[1]
        channels = frame.shape[2]
        # Flip the frame horizontally
        frame.flags.writeable = False
        frame = cv2.flip(frame, 1)
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
#         Get hand landmark prediction
        result = hands.process(frame)
    
#         Get face landmark prediction
        resultFace = face_mesh.process(frame)

        # post process the result
        frame.flags.writeable = True
#         frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            
        if result.multi_hand_landmarks:
            landmarks = []
            count = 0
            for handslms in result.multi_hand_landmarks:
                count +=1
                for lm in handslms.landmark:
                    # print(id, lm)
                    lmx = int(lm.x * x)
                    lmy = int(lm.y * y)

                    landmarks.append([lmx, lmy])

                # Drawing landmarks on frames
                if show_hand_lm:
                    mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)
                
                if (count <= 1) and predict_hand:
                    # Predict gesture
                    prediction_hand = model_hand.predict([landmarks], verbose =0)
                    # print(prediction)
                    classID_hand = np.argmax(prediction_hand)
                    className_hand = classNames_hand[classID_hand]
                    cv2.putText(frame, className_hand, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                                   1, (0,0,255), 2, cv2.LINE_AA)
#                 print("hands landmarks: ")
#                 display(handslms)
#         478 face mesh points
        if resultFace.multi_face_landmarks:
            for face_landmarks in resultFace.multi_face_landmarks:
#             current_time = datetime.datetime.now(datetime.timezone.utc)
                xyz = [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark]
                points = np.array(xyz)
        
                #Crop face from frame
                min_x = (min(points[:, 0])-EXTRA_PADDING) if (min(points[:, 0])-EXTRA_PADDING) > 0 else 0
                max_x = (max(points[:, 0])+EXTRA_PADDING) if (min(points[:, 0])+EXTRA_PADDING) < 1 else 1
                min_y = (min(points[:, 1])-EXTRA_PADDING) if (min(points[:, 0])-EXTRA_PADDING) > 0 else 0
                max_y = (max(points[:, 1])+EXTRA_PADDING) if (min(points[:, 0])+EXTRA_PADDING) < 1 else 1
                face_Coor_min_x = int((min_x)*width)
                face_Coor_max_x = int((max_x)*width)
                face_Coor_min_y = int((min_y)*height)
                face_Coor_max_y = int((max_y)*height)
                if (face_Coor_max_x-face_Coor_min_x) < (face_Coor_max_y-face_Coor_min_y):
                    face_Coor_max_x=math.ceil((face_Coor_max_x+face_Coor_min_x)/2)+math.ceil((face_Coor_max_y-face_Coor_min_y)/2)
                    face_Coor_min_x=math.floor((face_Coor_max_x+face_Coor_min_x)/2)-math.ceil((face_Coor_max_y-face_Coor_min_y)/2)
                    if face_Coor_min_x < 0 :
                        face_Coor_min_x = 0
                    if face_Coor_max_x > width:
                        face_Coor_max_x = width
                elif (face_Coor_max_x-face_Coor_min_x) > (face_Coor_max_y-face_Coor_min_y):
                    face_Coor_max_y=math.ceil((face_Coor_max_y+face_Coor_min_y)/2)+math.ceil((face_Coor_max_x-face_Coor_min_x)/2)
                    face_Coor_min_y=math.floor((face_Coor_max_y+face_Coor_min_y)/2)-math.ceil((face_Coor_max_x-face_Coor_min_x)/2)
                    if face_Coor_min_y < 0 :
                        face_Coor_min_y = 0
                    if face_Coor_max_y > height:
                        face_Coor_max_y = height
                count =0
                while (face_Coor_max_x-face_Coor_min_x) != (face_Coor_max_y-face_Coor_min_y):
                    if (face_Coor_max_x-face_Coor_min_x) < (face_Coor_max_y-face_Coor_min_y):
                        if count%2==0:
                            face_Coor_max_x+=1
                        elif count%2==1:
                            face_Coor_min_x-=1
                    elif (face_Coor_max_x-face_Coor_min_x) > (face_Coor_max_y-face_Coor_min_y):
                        if count%2==0:
                            face_Coor_max_y+=1
                        elif count%2==1:
                            face_Coor_min_y-=1
                    count+=1
                face = frame[face_Coor_min_y:face_Coor_max_y, face_Coor_min_x:face_Coor_max_x]

                #Resize to model input
                dim = (IMAGE_SIZE, IMAGE_SIZE)
                try:
                    resized = cv2.resize(face, dim, interpolation = cv2.INTER_AREA)
                except:
                    continue
                face=np.array([resized]) 
                
                if predict_face:
                    prediction = model_face.predict(face[:1], verbose='0')
                    classID = np.argmax(prediction)
                    className_face = classNames_face[classID]
                    cv2.putText(frame, className_face, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                           1, (0,0,255), 2, cv2.LINE_AA)

                if show_face_mesh:
                    mpDraw.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles
                        .get_default_face_mesh_tesselation_style())
                    mpDraw.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_CONTOURS,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles
                        .get_default_face_mesh_contours_style())
                    mpDraw.draw_landmarks(
                        image=frame,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_IRISES,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles
                        .get_default_face_mesh_iris_connections_style())
#             face_lm_inputs = 
#             display(xyzt)
        else:
            if predict_face:
                cv2.putText(frame, "No Face", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 
                           1, (0,0,255), 2, cv2.LINE_AA)
#         time.sleep(5)
        
        # show the prediction on the frame
#         frame = cv2.flip(frame, 1)
        
        
        
        # Show the final output
        
        
        keypress = cv2.waitKey(1) & 0xFF
        if keypress == ord('q'):
            break
        elif keypress == ord('f'):
            predict_face = not predict_face
        elif keypress == ord('m'):
            show_face_mesh = not show_face_mesh
        elif keypress == ord('h'):
            predict_hand = not predict_hand
            print("predict hand landmarks" + str(predict_hand))
        elif keypress == ord('c'):
            show_hand_lm = not show_hand_lm
            print("show hand landmarks" + str(show_hand_lm))
        elif keypress == ord(' '):
            gestures.append(className_face)
            print(gestures)
        elif keypress == ord('p'):
            for gesture in gestures:
                engine = pyttsx3.init()
                engine.say(gesture)
                engine.runAndWait()
                
        elif keypress == ord('d'):
            gestures = []
        
        position_x = 10
        for gesture in gestures:
            if gestures:
                cv2.putText(frame, gesture, (position_x, height-20), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.5, (0,0,255), 1, cv2.LINE_AA)
                position_x = position_x + 90
        
#         frame = cv2.resize(frame, (1280, 720), interpolation = cv2.INTER_AREA)
        cv2.imshow("Output", frame) 

# release the webcam and destroy all active windows
cap.release()

cv2.destroyAllWindows()